In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from keras.applications import ResNet101
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define your dataset directories
train_data_dir = "/content/drive/MyDrive/Models/train"
validation_data_dir = "/content/drive/MyDrive/Models/val"
batch_size = 32
target_size = (224, 224)

# Load pre-trained ResNet101 model with top (fully connected) layers
base_model = ResNet101(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

num_classes = 20  # Number of classes

# Create a custom top for the model
x = base_model.layers[-2].output  # Use the second-to-last layer as the base
x = layers.Dense(num_classes, activation='softmax')(x)

# Create the final fine-tuned model
model = models.Model(inputs=base_model.input, outputs=x)

# Compile the model with all requested metrics
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.AUC(name='auc')])

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001, restore_best_weights=True)

# Fine-tune the model with early stopping
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=500,  # Use a large number, early stopping will handle it
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    # callbacks=[early_stopping]
)

# Save the fine-tuned model
model.save('fine_tuned_resnet101.h5')

# Evaluate the model on the validation set
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)

# Convert predictions to class indices
y_pred_classes = tf.argmax(y_pred, axis=1).numpy()

# Calculate additional metrics
classification_report_str = classification_report(y_true, y_pred_classes, target_names=validation_generator.class_indices.keys(), zero_division=1)

# Print the classification report
print(classification_report_str)